In [1]:
# --- Block 5: Walk-Forward Cross-Validation + Hyperparameter-Search ---
import os, sys, json, time
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

In [2]:
# ---- Config laden ------------------------------------------------------
with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER, START, END, INTERVAL = C["ticker"], C["start"], C["end"], C["interval"]
HORIZON  = int(C["horizon"])
LOOKBACK_DEFAULT = int(C["lookback"])  # kann im Grid überschrieben werden
BATCH    = int(C.get("batch", 64))
EPOCHS   = int(C.get("epochs", 60))
SEED     = int(C.get("seed", 42))
FEATURESET = C.get("featureset", "v2")  # "v1" oder "v2"

np.random.seed(SEED)
import tensorflow as tf
tf.random.set_seed(SEED)

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RUN_DIR = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_wfcv")
(RUN_DIR / "plots").mkdir(parents=True, exist_ok=True)
print("WFCV_RUN_DIR:", RUN_DIR)

WFCV_RUN_DIR: ..\results\2025-10-14_17-49-33_wfcv


In [3]:
# ---- Daten & Features --------------------------------------------------
TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}.csv"
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=True).sort_index()

OHLCV = {"open","high","low","close","volume"}

# Feature-Liste aus YAML ziehen (v1/v2), fallback: alle nicht-OHLCV/target
import yaml, os
yaml_path = f"../data/features_{FEATURESET}.yml"
if os.path.exists(yaml_path):
    with open(yaml_path, "r") as f:
        meta = yaml.safe_load(f) or {}
    FEATURES_ALL = [c for c in meta.get("features", []) if c in df.columns]
else:
    FEATURES_ALL = [c for c in df.columns if c not in (OHLCV | {"target"})]

assert len(FEATURES_ALL) > 0, "Keine Features gefunden."
X_full = df[FEATURES_ALL].copy()
y_full = df["target"].astype(int).copy()

In [4]:
# ---- Walk-Forward Splits -----------------------------------------------
def make_wf_splits(n, n_folds=5, val_frac=0.15, min_train_frac=0.50):
    """Rolling WF: Train = [0:train_end], Val = (train_end:train_end+val_len].
       Schritte so gewählt, dass wir n_folds valide Fenster bekommen."""
    val_len = max(60, int(n * val_frac))     # min ~3 Monate (bei 1d ~60)
    min_train = max(200, int(n * min_train_frac))
    stops = []
    # letztes Val-Ende darf nicht über n hinausgehen
    last_val_end = n
    # wir verteilen n_folds Stopps gleichmäßig zwischen [min_train+val_len, n]
    span = last_val_end - (min_train + val_len)
    if span < n_folds:
        n_folds = max(1, span // max(1, val_len//2))
    for k in range(n_folds):
        val_end = min_train + val_len + int((k+1) * span / (n_folds))
        train_end = val_end - val_len
        train_slice = slice(0, train_end)             # [0, train_end)
        val_slice   = slice(train_end, val_end)       # [train_end, val_end)
        stops.append((train_slice, val_slice))
    return stops

n = len(df)
splits = make_wf_splits(n, n_folds=int(C.get("wfcv_folds", 5)))
print("Anzahl Folds:", len(splits))

Anzahl Folds: 5


In [5]:
# ---- Hilfsfunktionen: Windowing + Pipeline -----------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    balanced_accuracy_score, matthews_corrcoef,
    average_precision_score, roc_auc_score
)

def make_windows(X_df, y_ser, lookback):
    Xv = X_df.values.astype(np.float32)
    yv = y_ser.values.astype(np.int32)
    xs, ys = [], []
    for i in range(lookback-1, len(X_df)):
        xs.append(Xv[i - lookback + 1 : i + 1])
        ys.append(yv[i])
    return np.stack(xs, axis=0), np.array(ys)

from tensorflow.keras import layers, regularizers, callbacks, optimizers, models

def build_model(n_features, width1=64, width2=32, dropout=0.10, lr=5e-4, use_gru=True):
    rnn = layers.GRU if use_gru else layers.LSTM
    m = models.Sequential([
        layers.Input(shape=(None, n_features)),
        rnn(width1, return_sequences=True, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        rnn(width2, recurrent_dropout=dropout),
        layers.LayerNormalization(),
        layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-5)),
        layers.Dense(1, activation="sigmoid"),
    ])
    m.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC(name="auc"),
                 tf.keras.metrics.AUC(name="auprc", curve="PR")]
    )
    return m

def fit_eval_fold(X_tr, y_tr, X_va, y_va, lookback, hp, seed=SEED, batch=BATCH, epochs=EPOCHS):
    # Scaler NUR auf Train
    scaler = StandardScaler()
    X_tr_s = pd.DataFrame(scaler.fit_transform(X_tr), index=X_tr.index, columns=X_tr.columns)
    X_va_s = pd.DataFrame(scaler.transform(X_va),     index=X_va.index, columns=X_va.columns)

    Xtr_win, ytr = make_windows(X_tr_s, y_tr, lookback)
    Xva_win, yva = make_windows(X_va_s, y_va, lookback)

    ds_tr = tf.data.Dataset.from_tensor_slices((Xtr_win, ytr)).shuffle(len(Xtr_win), seed=seed).batch(batch).prefetch(tf.data.AUTOTUNE)
    ds_va = tf.data.Dataset.from_tensor_slices((Xva_win, yva)).batch(batch).prefetch(tf.data.AUTOTUNE)

    tf.keras.utils.set_random_seed(seed)
    model = build_model(
        n_features=Xtr_win.shape[-1],
        width1=hp["width1"], width2=hp["width2"],
        dropout=hp["dropout"], lr=hp["lr"], use_gru=hp["cell"]=="GRU"
    )

    cbs = [
        callbacks.EarlyStopping(monitor="val_auprc", mode="max", patience=8, restore_best_weights=True),
        callbacks.ReduceLROnPlateau(monitor="val_auprc", mode="max", factor=0.5, patience=4, min_lr=1e-5),
    ]
    hist = model.fit(ds_tr, validation_data=ds_va, epochs=epochs, verbose=0, callbacks=cbs)

    # Val-Probas → Scores
    yva_proba = model.predict(ds_va, verbose=0).ravel()
    yva_pred  = (yva_proba >= 0.5).astype(int)

    metrics = dict(
        mcc = float(matthews_corrcoef(yva, yva_pred)),
        bal_acc = float(balanced_accuracy_score(yva, yva_pred)),
        auprc = float(average_precision_score(yva, yva_proba)),
        auroc = float(roc_auc_score(yva, yva_proba)),
        epochs_trained = int(len(hist.history["loss"]))
    )
    return metrics

In [6]:
# ---- Hyperparameter-Grid ------------------------------------------------
# (klein halten, damit es zügig läuft; später erweiterbar)
LOOKBACK_GRID = [30, 60, max(90, LOOKBACK_DEFAULT)]
HP_GRID = [
    # width1, width2, dropout, lr, cell
    dict(width1=32, width2=16, dropout=0.10, lr=5e-4, cell="GRU"),
    dict(width1=64, width2=32, dropout=0.10, lr=5e-4, cell="GRU"),
    dict(width1=64, width2=32, dropout=0.20, lr=3e-4, cell="GRU"),
    dict(width1=64, width2=32, dropout=0.10, lr=5e-4, cell="LSTM"),
]

In [7]:
# Optional: Feature-Subsets ausprobieren (z. B. nur Momentum vs. alles)
FEATURE_SUBSETS = {
    "all": FEATURES_ALL,
    "mom_only": [c for c in FEATURES_ALL if "logret" in c or "sma_diff" in c or "macd" in c],
}

In [8]:
# ---- Suche ---------------------------------------------------------------
records = []
for feat_name, FEATS in FEATURE_SUBSETS.items():
    # sichere Minimalanzahl Features
    if len(FEATS) == 0: 
        continue
    for lookback in LOOKBACK_GRID:
        for hp in HP_GRID:
            fold_id = 0
            for tr_s, va_s in splits:
                fold_id += 1
                X_tr, y_tr = X_full.iloc[tr_s][FEATS], y_full.iloc[tr_s]
                X_va, y_va = X_full.iloc[va_s][FEATS], y_full.iloc[va_s]

                # genug Daten für Windowing?
                if len(X_tr) < (lookback + 50) or len(X_va) < (lookback + 10):
                    # überspringen, zu wenig Daten für diesen lookback
                    continue

                mets = fit_eval_fold(X_tr, y_tr, X_va, y_va, lookback, hp)

                rec = {
                    "feature_set": FEATURESET,
                    "features_used": feat_name,
                    "n_features": len(FEATS),
                    "lookback": lookback,
                    **hp,
                    "fold": fold_id,
                    **mets
                }
                records.append(rec)
                print(f"[{feat_name} | LB={lookback} | {hp['cell']} {hp['width1']}/{hp['width2']} dp={hp['dropout']} lr={hp['lr']}] "
                      f"Fold{fold_id}: MCC={mets['mcc']:.3f} AUPRC={mets['auprc']:.3f}")

results = pd.DataFrame.from_records(records)
csv_path = RUN_DIR / "wfcv_results.csv"
results.to_csv(csv_path, index=False)
print("\nGeschrieben:", csv_path)

[all | LB=30 | GRU 32/16 dp=0.1 lr=0.0005] Fold1: MCC=0.048 AUPRC=0.563
[all | LB=30 | GRU 32/16 dp=0.1 lr=0.0005] Fold2: MCC=0.015 AUPRC=0.529
[all | LB=30 | GRU 32/16 dp=0.1 lr=0.0005] Fold3: MCC=0.074 AUPRC=0.541
[all | LB=30 | GRU 32/16 dp=0.1 lr=0.0005] Fold4: MCC=-0.043 AUPRC=0.528
[all | LB=30 | GRU 32/16 dp=0.1 lr=0.0005] Fold5: MCC=-0.027 AUPRC=0.538
[all | LB=30 | GRU 64/32 dp=0.1 lr=0.0005] Fold1: MCC=0.038 AUPRC=0.575
[all | LB=30 | GRU 64/32 dp=0.1 lr=0.0005] Fold2: MCC=-0.045 AUPRC=0.532
[all | LB=30 | GRU 64/32 dp=0.1 lr=0.0005] Fold3: MCC=0.024 AUPRC=0.528
[all | LB=30 | GRU 64/32 dp=0.1 lr=0.0005] Fold4: MCC=0.050 AUPRC=0.538
[all | LB=30 | GRU 64/32 dp=0.1 lr=0.0005] Fold5: MCC=-0.071 AUPRC=0.527
[all | LB=30 | GRU 64/32 dp=0.2 lr=0.0003] Fold1: MCC=0.029 AUPRC=0.543
[all | LB=30 | GRU 64/32 dp=0.2 lr=0.0003] Fold2: MCC=-0.045 AUPRC=0.541
[all | LB=30 | GRU 64/32 dp=0.2 lr=0.0003] Fold3: MCC=0.054 AUPRC=0.532
[all | LB=30 | GRU 64/32 dp=0.2 lr=0.0003] Fold4: MCC=0.047

In [9]:
# ---- Aggregation & Best-Config -----------------------------------------
agg_cols = ["feature_set","features_used","n_features","lookback","width1","width2","dropout","lr","cell"]
agg = (results.groupby(agg_cols)
       .agg(mcc_mean=("mcc","mean"), mcc_std=("mcc","std"),
            auprc_mean=("auprc","mean"), auprc_std=("auprc","std"),
            auroc_mean=("auroc","mean"),
            balacc_mean=("bal_acc","mean"),
            n_folds=("mcc","count"))
       .reset_index())

# Primär: mcc_mean, Sekundär: auprc_mean
agg = agg.sort_values(["mcc_mean","auprc_mean"], ascending=False)
agg.to_csv(RUN_DIR / "wfcv_results_agg.csv", index=False)

best = agg.iloc[0].to_dict()
with open(RUN_DIR / "best_config.json", "w") as f:
    json.dump(best, f, indent=2)
print("Best config:", best)

Best config: {'feature_set': 'v2', 'features_used': 'all', 'n_features': 11, 'lookback': 60, 'width1': 32, 'width2': 16, 'dropout': 0.1, 'lr': 0.0005, 'cell': 'GRU', 'mcc_mean': 0.040597824839848376, 'mcc_std': 0.04491555611922093, 'auprc_mean': 0.5415718346484072, 'auprc_std': 0.021023191976759205, 'auroc_mean': 0.5125579801406488, 'balacc_mean': 0.5173959740529609, 'n_folds': 5}


In [10]:
# ---- einfache Score-Grids als Plot -------------------------------------
pivot_mcc = agg.pivot_table(index="lookback",
                            columns=["features_used", "cell", "width1"],
                            values="mcc_mean")
pivot_au  = agg.pivot_table(index="lookback",
                            columns=["features_used", "cell", "width1"],
                            values="auprc_mean")

for name, pivot in [("score_grid_mcc.png", pivot_mcc), ("score_grid_auprc.png", pivot_au)]:
    plt.figure(figsize=(10,5))
    im = plt.imshow(pivot.values, aspect="auto")
    plt.colorbar(im)
    plt.yticks(range(len(pivot.index)), pivot.index)
    plt.xticks(range(pivot.shape[1]), [str(c) for c in pivot.columns], rotation=45, ha="right")
    plt.title(name.replace("_"," ").replace(".png",""))
    plt.tight_layout()
    plt.savefig(RUN_DIR / "plots" / name, dpi=160)
    plt.close()

print("\nBlock 5 abgeschlossen. Artefakte:")
print(" -", RUN_DIR / "wfcv_results.csv")
print(" -", RUN_DIR / "wfcv_results_agg.csv")
print(" -", RUN_DIR / "best_config.json")
print(" -", RUN_DIR / "plots")


Block 5 abgeschlossen. Artefakte:
 - ..\results\2025-10-14_17-49-33_wfcv\wfcv_results.csv
 - ..\results\2025-10-14_17-49-33_wfcv\wfcv_results_agg.csv
 - ..\results\2025-10-14_17-49-33_wfcv\best_config.json
 - ..\results\2025-10-14_17-49-33_wfcv\plots
